In [13]:
from mp_api.client import MPRester
import inspect

try:
    with MPRester() as mpr:
        # 获取 search 方法的签名
        signature = inspect.signature(mpr.materials.summary.search)
        print("mpr.materials.summary.search 方法接受的参数:")
        for param in signature.parameters.values():
            print(f"- {param.name}: {param.kind} (default: {param.default})")

except Exception as e:
    print(f"检查方法签名时发生错误: {e}")
    print("请确保 mp-api 库已正确安装且 API Key 有效。")

mpr.materials.summary.search 方法接受的参数:
- band_gap: POSITIONAL_OR_KEYWORD (default: None)
- chemsys: POSITIONAL_OR_KEYWORD (default: None)
- crystal_system: POSITIONAL_OR_KEYWORD (default: None)
- density: POSITIONAL_OR_KEYWORD (default: None)
- deprecated: POSITIONAL_OR_KEYWORD (default: None)
- e_electronic: POSITIONAL_OR_KEYWORD (default: None)
- e_ionic: POSITIONAL_OR_KEYWORD (default: None)
- e_total: POSITIONAL_OR_KEYWORD (default: None)
- efermi: POSITIONAL_OR_KEYWORD (default: None)
- elastic_anisotropy: POSITIONAL_OR_KEYWORD (default: None)
- elements: POSITIONAL_OR_KEYWORD (default: None)
- energy_above_hull: POSITIONAL_OR_KEYWORD (default: None)
- equilibrium_reaction_energy: POSITIONAL_OR_KEYWORD (default: None)
- exclude_elements: POSITIONAL_OR_KEYWORD (default: None)
- formation_energy: POSITIONAL_OR_KEYWORD (default: None)
- formula: POSITIONAL_OR_KEYWORD (default: None)
- g_reuss: POSITIONAL_OR_KEYWORD (default: None)
- g_voigt: POSITIONAL_OR_KEYWORD (default: None)
- g_v

In [72]:
import pandas as pd
from mp_api.client import MPRester
from pymatgen.core import Composition, Element

YOUR_API_KEY = "" # 请确保这是您的真实API Key

print("--- 步骤 1: 从 Materials Project API 获取初步数据（包含 nsites 和 composition 字段） ---")
with MPRester(YOUR_API_KEY) as mpr:
    docs = mpr.materials.summary.search(
        crystal_system="Cubic",
        num_elements=[3, 4, 5],
        fields=[
            'material_id',
            'formula_pretty',
            'formation_energy_per_atom',
            'nelements',
            'elements',
            'symmetry',
            'nsites',
            'composition'
        ]
    )

print(f"初步从API获取了 {len(docs)} 个符合'Cubic'和'3,4或5种元素'条件的条目。")

data_list = []
for doc in docs:
    row = {
        'material_id': doc.material_id,
        'pretty_formula': doc.formula_pretty,
        'formation_energy_per_atom': doc.formation_energy_per_atom,
        'num_elements': doc.nelements,
        'elements': doc.elements,
        'crystal_system': doc.symmetry.crystal_system if doc.symmetry else None,
        'space_group_symbol': doc.symmetry.symbol if doc.symmetry else None,
        'nsites': doc.nsites,
        'composition': doc.composition
    }
    data_list.append(row)

df_preliminary = pd.DataFrame(data_list)
print(f"已将初步数据转换为DataFrame，共 {len(df_preliminary)} 条记录。")

halide_elements_str_set = {"F", "Cl", "Br", "I"}
# A位元素集合 (用于 B位元素筛选，尽管这个函数目前不使用了，但保留以防后续需要)
a_site_elements_str_set = {
    "Li", "Na", "K", "Rb", "Cs", "Fr", # 碱金属
    "Be", "Mg", "Ca", "Sr", "Ba", "Ra"  # 碱土金属
}

def get_num_halogens(composition_data):
    total_halogens = 0
    if isinstance(composition_data, Composition):
        comp_dict = composition_data.as_dict()
    elif isinstance(composition_data, dict):
        comp_dict = composition_data
    else:
        return 0
    for element_symbol, count in comp_dict.items():
        if element_symbol in halide_elements_str_set:
            total_halogens += count
    return total_halogens

def get_num_unique_halogen_types(elements_data):
    if not isinstance(elements_data, (list, tuple)):
        return 0
    found_halogens = set()
    for elem_obj in elements_data:
        elem_symbol = str(elem_obj)
        if elem_symbol in halide_elements_str_set:
            found_halogens.add(elem_symbol)
    return len(found_halogens)

# --- 验证 pretty_formula 的原子总数和卤素原子数（继续排除 ABX3 简化形式） ---
def validate_formula_stoichiometry(pretty_formula_str):
    try:
        comp = Composition(pretty_formula_str)
        formula_nsites = comp.num_atoms # 简化后的总原子数
        formula_num_halogens = 0 # 简化后的卤素原子数
        for element_symbol, count in comp.as_dict().items():
            if element_symbol in halide_elements_str_set:
                formula_num_halogens += count
        
        # 严格要求 pretty_formula 对应的总原子数是 10 且卤素数是 6
        # 这将排除 CsTiCl3 (因为它的 formula_nsites=5, formula_num_halogens=3)
        if formula_nsites == 10 and formula_num_halogens == 6:
            return True
        else:
            return False
            
    except Exception:
        return False

# --- 新增函数：检查 pretty_formula 字符串尾部是否是数字 '6' ---
def check_formula_ends_with_6(pretty_formula_str):
    if isinstance(pretty_formula_str, str) and pretty_formula_str.endswith('6'):
        return True
    return False


df_preliminary['num_halogens'] = df_preliminary['composition'].apply(get_num_halogens)
df_preliminary['num_unique_halogen_types'] = df_preliminary['elements'].apply(get_num_unique_halogen_types)
print(f"已计算每个材料的卤素原子总数和唯一卤素类型数量。")

print("\n--- 步骤 2: 在 Pandas DataFrame 中进行精确筛选 ---")

df_filtered_sg = df_preliminary[
    df_preliminary['space_group_symbol'].isin(["Fm-3m", "Pm-3m"])
].copy()
print(f"筛选出空间群为 'Fm-3m, Pm-3m' 的材料后，剩余 {len(df_filtered_sg)} 个。")

df_filtered_hal_types = df_filtered_sg[
    df_filtered_sg['num_unique_halogen_types'] <= 2
].copy()
print(f"进一步筛选出卤素类型不超过两种的材料后，剩余 {len(df_filtered_hal_types)} 个。")

df_filtered_nsites = df_filtered_hal_types[
    df_filtered_hal_types['nsites'] == 10
].copy()
print(f"进一步筛选出原子总数为 10 的材料后，剩余 {len(df_filtered_nsites)} 个。")

df_filtered_hal_count = df_filtered_nsites[
    df_filtered_nsites['num_halogens'] == 6
].copy()
print(f"进一步筛选出卤素原子个数为 6 的材料后，剩余 {len(df_filtered_hal_count)} 个。")

# --- 新增筛选条件 F: 严格验证 pretty_formula 是否符合 A2B'B''X6 (10原子, 6卤素) 的简化形式 ---
# 这一步会排除 CsTiCl3 (因为它pretty_formula解析后是5个原子)
df_final_strict_formula = df_filtered_hal_count[
    df_filtered_hal_count['pretty_formula'].apply(validate_formula_stoichiometry)
].copy()
print(f"进一步筛选出 pretty_formula 严格符合预期 A2B'B''X6 (10原子, 6卤素) 的材料后，剩余 {len(df_final_strict_formula)} 个。")

# --- 新增筛选条件 G: 检查 pretty_formula 字符串是否以 '6' 结尾 ---
# 这一步将作为额外的，您指定的筛选条件
df_final_ends_with_6 = df_final_strict_formula[
    df_final_strict_formula['pretty_formula'].apply(check_formula_ends_with_6)
].copy()
print(f"进一步筛选出 pretty_formula 字符串以 '6' 结尾的材料后，剩余 {len(df_final_ends_with_6)} 个。")


# 筛选条件 E: 稳定性（使用 formation_energy_per_atom 小于 0 作为判断）
df_final = df_final_ends_with_6[ # 注意这里是从 df_final_ends_with_6 开始筛选
    (df_final_ends_with_6['formation_energy_per_atom'].notna()) &
    (df_final_ends_with_6['formation_energy_per_atom'] < 0)
].copy()
print(f"最终筛选出形成能为负的稳定材料后，剩余 {len(df_final)} 个。")

if not df_final.empty:
    print("\n--- 最终符合所有条件的材料预览 (前5行): ---")
    print(df_final.head())
    print(f"\n成功找到 {len(df_final)} 个符合所有条件的双钙钛矿卤化物。")
    df_final.to_csv("filtered_perovskite_halides_strict_formula_ends_with_6.csv", index=False)
    print("数据已保存到 'filtered_perovskite_halides_strict_formula_ends_with_6.csv'。")
else:
    print("\n抱歉，经过所有筛选条件，未找到任何符合条件的材料。")
    print("这可能意味着数据库中同时满足所有严格条件的材料非常稀少，或者您需要稍微放宽某些条件。")

--- 步骤 1: 从 Materials Project API 获取初步数据（包含 nsites 和 composition 字段） ---


Retrieving SummaryDoc documents: 100%|█████████████████████████████████████████| 14562/14562 [00:07<00:00, 1986.90it/s]


初步从API获取了 14562 个符合'Cubic'和'3,4或5种元素'条件的条目。
已将初步数据转换为DataFrame，共 14562 条记录。
已计算每个材料的卤素原子总数和唯一卤素类型数量。

--- 步骤 2: 在 Pandas DataFrame 中进行精确筛选 ---
筛选出空间群为 'Fm-3m, Pm-3m' 的材料后，剩余 9678 个。
进一步筛选出卤素类型不超过两种的材料后，剩余 9673 个。
进一步筛选出原子总数为 10 的材料后，剩余 2979 个。
进一步筛选出卤素原子个数为 6 的材料后，剩余 2250 个。
进一步筛选出 pretty_formula 严格符合预期 A2B'B''X6 (10原子, 6卤素) 的材料后，剩余 2225 个。
进一步筛选出 pretty_formula 字符串以 '6' 结尾的材料后，剩余 2212 个。
最终筛选出形成能为负的稳定材料后，剩余 2212 个。

--- 最终符合所有条件的材料预览 (前5行): ---
     material_id pretty_formula  formation_energy_per_atom  num_elements  \
123   mp-1120715       Ag3BiBr6                  -0.492092             3   
255   mp-1205591        AlAg3F6                  -2.134455             3   
2003  mp-1205935        FeAg3F6                  -1.691776             3   
2080  mp-1207208        GaAg3F6                  -1.734441             3   
2326  mp-1079742        GeN2OF6                  -0.208248             4   

           elements crystal_system space_group_symbol  nsites    composition  \
123    [Ag, B